In [1]:
# code used to conduct paired t tests for each monitoring station
# Geography 176C Team Perfect Skies: Ethan Kadiyala, Mikal De Wit, Xiomara Mendez
# Code by Ethan Kadiyala, 6/4/2020

In [2]:
## Imports
import numpy as np
from scipy import stats
import pandas as pd

In [3]:
# import csv's, each csv contains data from a specific study region for the months of March and April
# set variables for each csv
bay_base= pd.read_csv('air_data/17_18_19_OZONE_BAY.csv')
la_base= pd.read_csv('air_data/17_18_19_OZONE_LA.csv')
la_2020= pd.read_csv('air_data/2020_OZONE_LA.csv')
bay_2020= pd.read_csv('air_data/2020_OZONE_BAY.csv')

In [4]:
# bay_base.head()
# la_base.head()
# bay_2020.head()
# la_2020.head()
# bay_base.columns

## Los Angeles Stations

In [5]:
# set simple variables for the dfs, LA first
df_old = la_base    # baseline from 2017-2019

df_new = la_2020    # 2020 data

In [6]:
# set the index as the site ID
df_old_ID = df_old.set_index(['ID'])

df_new_ID = df_new.set_index(['ID'])


In [7]:
# make sure the dfs contain the same sites
df_old_ID_match = df_old_ID.loc[df_new_ID.index.unique()]

In [8]:
print(df_old_ID_match.index.unique())

Int64Index([60370002, 60370016, 60370113, 60371103, 60371201, 60371302,
            60371602, 60371701, 60372005, 60375005, 60376012, 60379033,
            60590007, 60592022, 60595001, 60650012, 60650016, 60656001,
            60658001, 60658005, 60659001, 60710005, 60710012, 60710306,
            60711004, 60712002, 60714001, 60714003, 60719004, 60731008,
            61110007, 61110009, 61112002, 61113001],
           dtype='int64', name='ID')


In [9]:
print(df_new_ID.index.unique())

Int64Index([60370002, 60370016, 60370113, 60371103, 60371201, 60371302,
            60371602, 60371701, 60372005, 60375005, 60376012, 60379033,
            60590007, 60592022, 60595001, 60650012, 60650016, 60656001,
            60658001, 60658005, 60659001, 60710005, 60710012, 60710306,
            60711004, 60712002, 60714001, 60714003, 60719004, 60731008,
            61110007, 61110009, 61112002, 61113001],
           dtype='int64', name='ID')


In [10]:
# reset index
df_old = df_old_ID_match.reset_index()

df_new = df_new_ID.reset_index()

# convert DATE to datetime objects
df_old['DATE'] = pd.to_datetime(df_old['DATE'])
df_new['DATE'] = pd.to_datetime(df_new['DATE'])

In [11]:
# set index to DATE
df_old_date = df_old.set_index('DATE')

df_new_date = df_new.set_index('DATE')
df_new_date.head()

,ID,DAY_MAX_OZONE,DAILY_AQI_VALUE,SITE_NAME,DAILY_OBS_COUNT,PERCENT_COMPLETE,CBSA_CODE,CBSA_NAME,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
DATE,,,,,,,,,,,
2020-03-01,60370002,0.044,41,Azusa,24,100,31080,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,34.1365,-117.92391
2020-03-02,60370002,0.042,39,Azusa,24,100,31080,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,34.1365,-117.92391
2020-03-03,60370002,0.042,39,Azusa,24,100,31080,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,34.1365,-117.92391
2020-03-04,60370002,0.039,36,Azusa,24,100,31080,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,34.1365,-117.92391
2020-03-05,60370002,0.048,44,Azusa,24,100,31080,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles,34.1365,-117.92391


In [12]:
# make an empty list 
df_list = []

# design a for loop that uses masking to iterate through the IDs, and reindexing to include all days in the month of march and april
for ID in df_new_date['ID'].unique():
    # mask by ID
    x = df_new_date[df_new_date.ID == ID]
    # set range for index
    idx = pd.date_range(min(df_new_date.index), max(df_new_date.index))
    # reindex the df to have 61 days
    y = x.reindex(idx)
    # make sure that the ID column is complete, no NaNs
    y['ID'] = ID
    # append to the newly created list
    df_list.append(y)
    
# concatenate the new list     
df_2020 = pd.concat(df_list)

In [13]:
# group baseline by site, then month, then day
baseline =  df_old_date.groupby([df_old_date.ID, df_old_date.index.month, df_old_date.index.day])['DAY_MAX_OZONE'].mean()
baseline
# convert to a df
df_base = pd.DataFrame(baseline)

# reset the index one level at a time
df_base_1 = df_base.reset_index(level = 0)
df_base_2 = df_base_1.reset_index(level = 0)

# rename DATE to DAY to avoid duplicate column names
df_base_2['DAY'] = df_base_2['DATE']
# delete the DATE column
del df_base_2['DATE']

# reset the last index
df_base_3 = df_base_2.reset_index()

# rename DATE to MONTH
df_base_3['MONTH'] = df_base_3['DATE']
# delete the DATE column
del df_base_3['DATE']

# simplify variable name
df_base = df_base_3

# a three year baseline of ozone data, contains daily data for 34 monitoring stations over 61 days each, totaling 2074  

In [14]:
# conduct t-test on all sites
for ID in df_base.ID.unique():
    x = df_base.DAY_MAX_OZONE[df_base.ID == ID]
    y = df_2020.DAY_MAX_OZONE[df_2020.ID == ID]
    test = stats.ttest_rel(x, y, nan_policy = 'omit')
    print(test, 'Station:', ID)


Ttest_relResult(statistic=2.037803602805772, pvalue=0.04613950281076289) Station: 60370002
Ttest_relResult(statistic=-0.28338164141463373, pvalue=0.7780094804173324) Station: 60370016
Ttest_relResult(statistic=5.401866970687692, pvalue=1.2391244579381627e-06) Station: 60370113
Ttest_relResult(statistic=5.338666585596079, pvalue=1.8320515980622785e-06) Station: 60371103
Ttest_relResult(statistic=3.0116862351050675, pvalue=0.003919834309321576) Station: 60371201
Ttest_relResult(statistic=4.513792293219869, pvalue=3.029843101235114e-05) Station: 60371302
Ttest_relResult(statistic=0.028196445188581182, pvalue=0.9776057182615434) Station: 60371602
Ttest_relResult(statistic=-3.953304901872027, pvalue=0.00020887431709252108) Station: 60371701
Ttest_relResult(statistic=-0.11454094200257547, pvalue=0.9092115200251386) Station: 60372005
Ttest_relResult(statistic=11.262200839131589, pvalue=5.170444837856985e-16) Station: 60375005
Ttest_relResult(statistic=1.041010271597503, pvalue=0.3022663025042

In [15]:
# find critical value

# set up columns to be added to a new df
df_base['OZONE_BASE'] = df_base['DAY_MAX_OZONE']

df_2020['OZONE_20'] = df_2020['DAY_MAX_OZONE']

# reset the index of the 2020 data
df_20 = df_2020.reset_index()

# make new df with ozone values from both years
ozone_20 = df_20['OZONE_20']
df_ozone_20 = pd.DataFrame(ozone_20)
df_ozone_20
# df with just the ozone data from both years
df_ozone = pd.concat([df_ozone_20, df_base['OZONE_BASE']], axis = 1)

In [16]:
# calculate critical value

# find the difference between the samples
df_ozone['d'] = df_ozone['OZONE_20'] - df_ozone['OZONE_BASE']

# calculate the mean of the differences, d bar
d_bar = df_ozone.d.mean() 

# claculate standard deviation of the differences
s_d = df_ozone.d.std()

# find n
n = len(df_ozone.d)

# t value for a one sided t test with 2073 degrees of freedom
t = 1.645589

# claculate statisitc to see if d_bar is significantly different
stat = d_bar*(s_d/(n**.5))

# calculate critical value for d_bar in order to yield significant results
crit = t*(s_d/(n**.5))
crit

0.0004012833956885438

## Bay Area Stations

In [17]:
# set simple variables for the Bay dfs
df_old = bay_base    # baseline from 2017-2019

df_new = bay_2020    # 2020 data

In [18]:
# set the index as the site ID
df_old_ID = df_old.set_index(['ID'])

df_new_ID = df_new.set_index(['ID'])


In [19]:
# make sure the dfs contain the same sites
df_old_ID_match = df_old_ID.loc[df_new_ID.index.unique()]

In [20]:
# reset index
df_old = df_old_ID_match.reset_index()

df_new = df_new_ID.reset_index()

# convert DATE to datetime objects
df_old['DATE'] = pd.to_datetime(df_old['DATE'])
df_new['DATE'] = pd.to_datetime(df_new['DATE'])

In [21]:
# set index to DATE
df_old_date = df_old.set_index('DATE')

df_new_date = df_new.set_index('DATE')

In [22]:
# make an empty list 
df_list = []

# design a for loop that uses masking to iterate through the IDs, and reindexing to include all days in the month of march and april
for ID in df_new_date['ID'].unique():
    # mask by ID
    x = df_new_date[df_new_date.ID == ID]
    # set range for index
    idx = pd.date_range(min(df_new_date.index), max(df_new_date.index))
    # reindex the df to have 61 days
    y = x.reindex(idx)
    # make sure that the ID column is complete, no NaNs
    y['ID'] = ID
    # append to the newly created list
    df_list.append(y)
    
# concatenate the new list     
df_2020 = pd.concat(df_list)

In [23]:
# group baseline by site, then month, then day
baseline =  df_old_date.groupby([df_old_date.ID, df_old_date.index.month, df_old_date.index.day])['DAY_MAX_OZONE'].mean()
baseline
# convert to a df
df_base = pd.DataFrame(baseline)

# reset the index one level at a time
df_base_1 = df_base.reset_index(level = 0)
df_base_2 = df_base_1.reset_index(level = 0)

# rename DATE to DAY to avoid duplicate column names
df_base_2['DAY'] = df_base_2['DATE']
# delete the DATE column
del df_base_2['DATE']

# reset the last index
df_base_3 = df_base_2.reset_index()

# rename DATE to MONTH
df_base_3['MONTH'] = df_base_3['DATE']
# delete the DATE column
del df_base_3['DATE']

# simplify variable name
df_base = df_base_3
 

In [24]:
# conduct t-test on all sites
for ID in df_base.ID.unique():
    x = df_base.DAY_MAX_OZONE[df_base.ID == ID]
    y = df_2020.DAY_MAX_OZONE[df_2020.ID == ID]
    if len(x) == 61:    
        t_stat, p_val = stats.ttest_rel(x, y, nan_policy = 'omit')
        print(p_val, 'Station:', ID)
    else:
        print(ID, 'is not equal length')

0.3618990112175112 Station: 60010007
0.2541666931217446 Station: 60010009
0.2671886542453742 Station: 60010011
0.016971421862392238 Station: 60010013
0.0002518698688271706 Station: 60012001
0.0603417921565984 Station: 60130002
0.8314655636300736 Station: 60131002
0.02059579079367137 Station: 60131004
0.0030223876584316457 Station: 60132007
0.0009486586967855398 Station: 60410001
0.023536071078299123 Station: 60550004
0.14625740425816997 Station: 60670011
0.7258197278247569 Station: 60750005
0.05446150095840288 Station: 60771002
0.00015371719514104622 Station: 60773005
0.0385934009086495 Station: 60811001
0.03601919273495009 Station: 60850005
6.083564142360394e-06 Station: 60851001
7.522098062839236e-05 Station: 60852006
0.2948229460635326 Station: 60950004
0.6209446758665926 Station: 60950005
60953002 is not equal length
0.7145415066838041 Station: 60970004


In [25]:
# find critical value

# set up columns to be added to a new df
df_base['OZONE_BASE'] = df_base['DAY_MAX_OZONE']

df_2020['OZONE_20'] = df_2020['DAY_MAX_OZONE']

# reset the index of the 2020 data
df_20 = df_2020.reset_index()

# make new df with ozone values from both years
ozone_20 = df_20['OZONE_20']
df_ozone_20 = pd.DataFrame(ozone_20)
df_ozone_20
# df with just the ozone data from both years
df_ozone = pd.concat([df_ozone_20, df_base['OZONE_BASE']], axis = 1)

In [26]:
# calculate critical value

# find the difference between the samples
df_ozone['d'] = df_ozone['OZONE_20'] - df_ozone['OZONE_BASE']

# calculate the mean of the differences, d bar
d_bar = df_ozone.d.mean() 

# claculate standard deviation of the differences
s_d = df_ozone.d.std()

# find n
n = len(df_ozone.d)

# t value for a one sided t test with 2073 degrees of freedom
t = 1.645589

# claculate statisitc to see if d_bar is significantly different
stat = d_bar*(s_d/(n**.5))

# calculate critical value for d_bar in order to yield significant results
crit = t*(s_d/(n**.5))
crit

0.0002856682474314262